In [3]:
import pickle
import os

In [65]:
input_dir = 'data/pickled_files/'
label_2_num_mapping = {}
num_2_label_ar= []
data = {"x":[],"y":[]}


In [70]:
label_num = 0
for label in os.listdir(input_dir):
    if label == '.DS_Store':
        continue
    p_file = input_dir + label
    label_2_num_mapping[label]=label_num
    num_2_label_ar.append(label)

    with open(p_file, 'rb') as f:
        content  = pickle.load(f)
        data['x'].extend(content)
        y_labels = [label_num for i in range(0,len(content))]
        data['y'].extend(y_labels)
    label_num+=1
          